In [ ]:
from spark_utils import quick_start
spark = quick_start("JupyterConnection")

In [ ]:
df = spark.read.format("delta").load("s3a://delta-lake/bronze/bronze_github_streaming_keyword_extractions")

In [ ]:
df.orderBy(df.processing_time.desc()).show(10)

In [ ]:
df.printSchema()

# Show first few rows
df.show(5)

# Get row count
print(f"Total rows: {df.count()}")

# Check columns
print(f"Columns: {df.columns}")

# Check processing_time column details
df.select("processing_time").describe().show()

# See latest records
df.orderBy(df.processing_time.desc()).show(10)

# Check time range
from pyspark.sql.functions import min, max
df.select(min("processing_time").alias("earliest"),
        max("processing_time").alias("latest")).show()

# Check how many records per minute (recent)
from pyspark.sql.functions import date_format, count

df.withColumn("minute", date_format("processing_time", "yyyy-MM-dd HH:mm")) \
.groupBy("minute") \
.count() \
.orderBy("minute", ascending=False) \
.show(20)

# Get last 1 minute of data
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.types import TimestampType

# Cast processing_time to timestamp if needed
df_recent = df.filter(
  col("processing_time").cast(TimestampType()) >=
  current_timestamp() - expr("INTERVAL 1 MINUTE")
)

print(f"Records in last minute: {df_recent.count()}")
df_recent.show()

# Top technologies in recent data
df.groupBy("keyword") \
.sum("mentions") \
.orderBy("sum(mentions)", ascending=False) \
.show(10)

In [ ]:
# See what hours actually have data
df.groupBy("date", "hour").count().orderBy("date", "hour").show(50)


# See most recent records and their hours
from pyspark.sql.functions import desc
df.select("date", "hour", "processing_time", "keyword") \
.orderBy(desc("processing_time")) \
.show(20)

# Check if there's data in hour 23 (current hour)
current_hour = 23
print(f"Records in hour {current_hour}:")
df.filter(f"hour = {current_hour}").count()

# Check hour 22 as well
print(f"Records in hour 22:")
df.filter(f"hour = 22").count()

# See the actual time range of your data
from pyspark.sql.functions import min, max, current_timestamp

df.select(
  min("processing_time").alias("earliest"),
  max("processing_time").alias("latest"),
  current_timestamp().alias("now")
).show()

# This might show partition info in the explain plan
df.explain()

# Or check the schema
df.printSchema()

In [ ]:
df = spark.read.format("delta").load("s3a://delta-lake/bronze/bronze_github_streaming_keyword_extractions")
df.createOrReplaceTempView("temp_keyword_streaming")

In [ ]:
spark.sql("""
DELETE FROM delta.`s3a://delta-lake/bronze/bronze_github_streaming_keyword_extractions`
WHERE date < DATE('2024-09-01')
""")

In [ ]:
df = spark.read.format("delta").load("s3a://delta-lake/bronze/bronze_github_streaming_keyword_extractions")
df.selectExpr("min(date)").show()

In [ ]:
df.schema

In [ ]:
df.show(5);

In [ ]:
df.groupBy("keyword").count().orderBy("count", ascending=False).show(10)